In [1]:
from OperacionesCSV import importar_CSV
from Neo4HungerGames import Neo4HungerGames
from Neo4Dataframes import Neo4Dataframes
from Neo4Graphics import Neo4Graphics
import pandas as pd
from flask import Flask, render_template, request, Response
from Controller import Controller
from Link_Weights import family_relationships_weights

import base64
def encode_image(response):
    return base64.b64encode(response.get_data()).decode("utf-8")

uri = "bolt://localhost:7687"
user = "mdad"
password = "123456789"

In [ ]:
neoConnection = Neo4HungerGames(uri, user, password)

app = Flask(__name__)
graphs = Neo4Graphics(neoConnection)
# Ruta principal
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/characters', methods=['GET'])
def characters():
    districts = neoConnection.get_all_districts()
    characters = neoConnection.get_some_characters()
    return render_template('characters.html', characters=characters, districts=districts)

@app.route('/characters/all', methods=['GET'])
def all_characters():
    characters = neoConnection.get_all_characters()
    return render_template('characters_list.html', characters=characters)

@app.route('/characters/create', methods=['POST'])
def create_character():
    return Controller.create_character(neoConnection, request=request)

@app.route("/characters/link_game", methods=["GET", "POST"])
def link_character_game():
    return Controller.link_character_game(neoConnection, request=request)

@app.route("/characters/update/<int:character_id>", methods=["GET"])
def edit_character_form(character_id):
    character = neoConnection.get_character_by_id(character_id)
    return render_template("update_character.html", character=character, character_id=character_id)

@app.route("/characters/update/<int:character_id>", methods=["POST"])
def update_character(character_id):
    return Controller.update_character(neoConnection, request, character_id)

@app.route("/characters/delete", methods=["POST"])
def delete_character():
    return Controller.delete_character(neoConnection, request)

@app.route("/characters/delete/<character_id>", methods=["POST"])
def delete_character_by_id(character_id):
    return Controller.delete_character_by_id(neoConnection, character_id)

@app.route("/characters/link_family", methods=["GET"])
def show_family_link_form():
    characters = neoConnection.get_all_characters_id_and_name()
    relation_types = family_relationships_weights.keys()
    return render_template("link_family.html", characters=characters, relation_types=relation_types)

@app.route("/characters/link_family", methods=["POST"])
def create_family_link():
    return Controller.create_family_link(neoConnection, request)

@app.route("/characters/family/all")
def get_all_family_links():
    return Controller.get_all_family_links(neoConnection)

@app.route("/characters/family/delete", methods=["POST"])
def delete_family_link():
    return Controller.delete_family_link(neoConnection, request.form)

@app.route("/links", methods=["GET"])
def list_links():
    return Controller.list_links(neoConnection)

@app.route("/links/delete", methods=["POST"])
def delete_link():
    return Controller.delete_link(neoConnection, request.form)

@app.route("/stats", methods=["GET"])
def stats():
    max_appearances = neoConnection.get_max_appearances()[0]

    books = encode_image(graphs.characters_per_book())
    districts = encode_image(graphs.characters_per_district())
    alliances = encode_image(graphs.characters_per_alliance())
    games = encode_image(graphs.characters_per_game())
    kills = encode_image(graphs.kills_per_cause())
    districts_per_book = encode_image(graphs.bubble_chart_districts_per_book())

    return render_template("stats.html",
                           max_appearances=max_appearances,
                           books=books,
                           districts=districts,
                           alliances=alliances,
                           games=games,
                           kills=kills,
                           districts_per_book=districts_per_book)

@app.route("/characters/other_books", methods=["GET", "POST"])
def characters_other_books():
    books = neoConnection.get_all_books()  # Para popular el select en HTML
    if request.method == "POST":
        title = request.form.get("title")
        characters = neoConnection.get_characters_from_other_books(title)
        return render_template("characters_other_books.html", books=books, characters=characters, selected_title=title)
    return render_template("characters_other_books.html", books=books, characters=None, selected_title=None)

@app.route("/books", methods=["GET", "POST"])
def book_characters():
    books = neoConnection.get_all_books()  # Para popular el select en HTML
    if request.method == "POST":
        title = request.form.get("title")
        characters = neoConnection.get_characters_from_a_book(title)
        return render_template("book_characters.html", books=books, characters=characters, selected_title=title)
    return render_template("book_characters.html", books=books, characters=None, selected_title=None)

@app.route("/deaths", methods=["GET", "POST"])
def dead_characters():
    death_causes = neoConnection.get_count_kills_per_cause()
    print(death_causes)
    if request.method == "POST":
        cause = request.form.get("cause")
        characters = neoConnection.get_dead_characters_from_a_cause(cause)
        return render_template("dead_characters.html", death_causes=death_causes, characters=characters, selected_cause=cause)
    return render_template("dead_characters.html", death_causes=death_causes, characters=None, selected_cause=None)

@app.route("/characters/other_districts", methods=["GET"])
def show_cross_district_connections():
    data = neoConnection.get_characters_and_their_known_people_from_other_districts()
    return render_template("known_people.html", characters=data)

@app.route("/shortest_path", methods=["GET", "POST"])
def show_shortest_path():
    characters = neoConnection.get_all_characters()
    path_data = None
    if request.method == "POST":
        source_id = request.form.get("source_id")
        target_id = request.form.get("target_id")
        source = neoConnection.get_character_by_id(source_id)
        target = neoConnection.get_character_by_id(target_id)
        dijkstra_data = neoConnection.shortest_path_dijkstra(source_id, target_id)
        path_data = neoConnection.shortest_path_jumps(source_id, target_id)
        return render_template("shortest_path.html", characters=characters, source_id=source_id, target_id=target_id, path_data=path_data, source=source, target=target, dijkstra_data=dijkstra_data)
    return render_template("shortest_path.html", characters=characters, source_id=None, target_id=None, path_data=None, source=None, target=None, dijkstra_data=None)

@app.route('/most_involved_game', methods=['GET'])
def show_game_with_most_involved():
    data = neoConnection.get_game_with_most_involved_characters_detailed()
    return render_template('most_involved_game.html', data=data)

@app.route('/mentor_victories')
def mentors_with_victories():
    mentors = neoConnection.get_mentors_with_victorious_tributes()
    return render_template('mentors_victories.html', mentors=mentors)

@app.route("/characters/<character_id>", methods=["GET"], endpoint='get_character_by_id')
def get_character_by_id(character_id):
    character = neoConnection.get_character_by_id(character_id)
    if character:
        return render_template("character_detail.html", characters=character)
    else:
        return "Personaje no encontrado", 404
    
@app.route("/characters/name", methods=["GET"])
def get_character_by_name():
    name = request.args.get("name")
    if not name:
        return "Nombre no proporcionado", 400
    characters = neoConnection.get_character_by_name(name)
    if characters:
        return render_template("character_detail.html", characters=characters)
    else:
        return "No se encontraron personajes", 404


if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=8080,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.33:8080
Press CTRL+C to quit
127.0.0.1 - - [18/May/2025 13:29:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2025 13:29:56] "GET /static/style.css HTTP/1.1" 304 -
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':ALLY_OF|BELONGS_TO' instead)} {position: line: 2, column: 46, offset: 46} for query: '\n                MATCH (c:Character)-[:ALLY_OF|:BELONGS_TO]->(a:Character|Alliance)\n                RETURN a.Name AS alliance, count(c) AS character_count\n                ORDER BY character_count DESC\n            '
Received notification from DBMS server: {severity: WARNI

In [ ]:
neoConnection.close()

In [ ]:
ruta_csv='HungerGames_Characters_Dataset_ALL.csv'

hgDF = importar_CSV(ruta_csv)
hgDF['ID'] = hgDF.index
dataConnection = Neo4Dataframes(uri, user, password)
dataConnection.delete_all_links_and_nodes()
dataConnection.create_nodes_links_from_df(hgDF)
dataConnection.close()